<a href="https://colab.research.google.com/github/jyoti246/S3DNN/blob/main/Copy_of_cuda_alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-8c4e2njt
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-8c4e2njt
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=5d849835507dc0a857182b48791de408621c77af84531f84fad08e5d0a807e80
  Stored in directory: /tmp/pip-ephem-wheel-cache-5entshwc/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
!git clone https://github.com/alessandrobessi/cuda-lab.git

Cloning into 'cuda-lab'...
remote: Enumerating objects: 56, done.
remote: Total 56 (delta 0), reused 0 (delta 0), pack-reused 56
Unpacking objects: 100% (56/56), done.


In [ ]:
!chmod 755 cuda-lab/INSTALL.sh

In [ ]:
!./cuda-lab/INSTALL.sh

35 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2021-04-12 19:35:26--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/8.0/prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb [following]
--2021-04-12 19:35:26--  https://developer.nvidia.com/compute/cuda/8.0/prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?ge8TEWN9FQ1

In [ ]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cuda --name alex.cu
#include <cudnn.h>
#include <cublas_v2.h>
#include <cuda_runtime.h>
#include <cstdlib>
#include <cassert>
#include <cstdlib>
#include <iostream>
#include <string>
#include <random>
#include <cmath>
#include <stdio.h>
#include <bits/stdc++.h>
#include <cuda_runtime.h>

using namespace std;

#define BATCH_SIZE 16
#define MAX_THREADS_PER_BLOCK 1024 // according to GTX 1050 Ti
#define LAYERS 5
#define COMP_TIME 3
#define EXEC_TIME 1
#define NUM_BATCHES 2

int roundUp(int num, int den)
{

  return((num + den - 1 )/(den));

}

struct convDim_t{

  int Height;
  int Width;
  int Channels;
  int Batch;
};

struct kernelDim_t{

  int kernelSize;
  int kernelHeight;
  int kernelWidth;
  int strideHeight;
  int strideWidth;
  int padHeight;
  int padWidth;
  int dilationHeight;
  int dilationWidth;
};


convDim_t setConvSpecs(int ht, int wd, int ch, int bt){

  convDim_t temp;
  temp.Height = ht;
  temp.Width = wd;
  temp.Channels = ch;
  temp.Batch = bt;

  return temp;
}

kernelDim_t setKernelSpecs(int size, int fheight, int fwidth, int sheight, int swidth, int pheight, int pwidth, int dheight, int dwidth){

  kernelDim_t layerKernel;
  layerKernel.kernelSize = size;
  layerKernel.kernelHeight = fheight;
  layerKernel.kernelWidth = fwidth;
  layerKernel.strideHeight = sheight;
  layerKernel.strideWidth = swidth;
  layerKernel.padHeight = pheight;
  layerKernel.padWidth = pwidth;
  layerKernel.dilationHeight = dheight;
  layerKernel.dilationWidth = dwidth;

  return layerKernel;
}



#define checkCUDNN(expression)                             \
{                                                          \
  cudnnStatus_t status = (expression);                     \
  if (status != CUDNN_STATUS_SUCCESS) {                    \
    std::cerr << "Error on line " << __LINE__ << ": "      \
              << cudnnGetErrorString(status) << std::endl; \
    std::exit(EXIT_FAILURE);                               \
  }                                                        \
}


float alpha = 1.0;
float beta = 0.0;

class ConvLayers{
    
public:
    float *kernelTensor{nullptr};		
    int layerIndex;
	  size_t workspace_bytes{0};

    convDim_t outDims;
    convDim_t inDims;
    kernelDim_t kernelDims;
	    

    cudnnTensorDescriptor_t input_descriptor;
    cudnnTensorDescriptor_t output_descriptor;
    cudnnFilterDescriptor_t kernel_descriptor;
    cudnnConvolutionDescriptor_t convolution_descriptor;
    cudnnConvolutionFwdAlgo_t convolution_algorithm;

    ConvLayers(){}

    ConvLayers(int index, convDim_t inDim, kernelDim_t kdims, convDim_t outDims){

      this->inDims = inDim;
      this->kernelDims = kdims;
      this->layerIndex = index;
	    this->outDims = outDims;
    }

    void set(int index, convDim_t inDim, kernelDim_t kdims, convDim_t outDims){

      this->inDims = inDim;
      this->kernelDims = kdims;
      this->layerIndex = index;
      this->outDims = outDims;
    }

    void buildConvLayer();

    void fwdProp(cudaStream_t stream, cudnnHandle_t cudnn, float *inputTensor, float* &outputTensor, void* &d_workspace);

};

void ConvLayers::buildConvLayer(){
	checkCUDNN(cudnnCreateTensorDescriptor(&input_descriptor));
	checkCUDNN(cudnnSetTensor4dDescriptor(input_descriptor,
										/*format=*/CUDNN_TENSOR_NHWC,
										/*dataType=*/CUDNN_DATA_FLOAT,
										/*batch_size=*/inDims.Batch,
										/*channels=*/inDims.Channels,
										/*image_height=*/inDims.Height,
										/*image_width=*/inDims.Width));



	checkCUDNN(cudnnCreateTensorDescriptor(&output_descriptor));
	checkCUDNN(cudnnSetTensor4dDescriptor(output_descriptor,
										/*format=*/CUDNN_TENSOR_NHWC,
										/*dataType=*/CUDNN_DATA_FLOAT,
										/*batch_size=*/outDims.Batch,
										/*channels=*/outDims.Channels,
										/*image_height=*/outDims.Height,
										/*image_width=*/outDims.Width));   
										
										

	checkCUDNN(cudnnCreateFilterDescriptor(&kernel_descriptor));
	checkCUDNN(cudnnSetFilter4dDescriptor(kernel_descriptor,
										/*dataType=*/CUDNN_DATA_FLOAT,
										/*format=*/CUDNN_TENSOR_NCHW,
										/*out_channels=*/outDims.Channels,
										/*in_channels=*/inDims.Channels,
										/*kernel_height=*/kernelDims.kernelHeight,
										/*kernel_width=*/kernelDims.kernelWidth)); 
										
										

	checkCUDNN(cudnnCreateConvolutionDescriptor(&convolution_descriptor));
	checkCUDNN(cudnnSetConvolution2dDescriptor(convolution_descriptor,
										/*pad_height=*/kernelDims.padHeight,
										/*pad_width=*/kernelDims.padWidth,
										/*vertical_stride=*/kernelDims.strideHeight,
										/*horizontal_stride=*/kernelDims.strideWidth,
										/*dilation_height=*/kernelDims.dilationHeight,
										/*dilation_width=*/kernelDims.dilationWidth,
										/*mode=*/CUDNN_CROSS_CORRELATION,
										/*computeType=*/CUDNN_DATA_FLOAT));


	float h_kernel[outDims.Channels][inDims.Channels][kernelDims.kernelHeight][kernelDims.kernelWidth];
    for (int kernel = 0; kernel < outDims.Channels; ++kernel) {
    for (int channel = 0; channel < inDims.Channels; ++channel) {
        for (int row = 0; row < kernelDims.kernelHeight; ++row) {
        for (int column = 0; column < kernelDims.kernelWidth; ++column) {
            h_kernel[kernel][channel][row][column] = 0.5;
        }
        }
    }
    }

    cudaMalloc(&kernelTensor, sizeof(h_kernel));
    cudaMemcpy(kernelTensor, h_kernel, sizeof(h_kernel), cudaMemcpyHostToDevice); 
}


void ConvLayers::fwdProp(cudaStream_t stream, cudnnHandle_t cudnn, float *inputTensor, float* &outputTensor, void* &d_workspace)
{
	checkCUDNN(cudnnGetConvolutionForwardAlgorithm(cudnn,
                                            input_descriptor,
                                            kernel_descriptor,
                                            convolution_descriptor,
                                            output_descriptor,
                                            CUDNN_CONVOLUTION_FWD_PREFER_FASTEST,
                                            /*memoryLimitInBytes=*/0,
                                            &convolution_algorithm));
                                            

    checkCUDNN(cudnnGetConvolutionForwardWorkspaceSize(cudnn,
                                                    input_descriptor,
                                                    kernel_descriptor,
                                                    convolution_descriptor,
                                                    output_descriptor,
                                                    convolution_algorithm,
                                                    &workspace_bytes));                  
                
    cudaMalloc(&d_workspace, workspace_bytes);       

	int out_bytes = outDims.Batch*outDims.Channels*outDims.Height*outDims.Width*sizeof(float);
    cudaMalloc(&outputTensor, out_bytes);
    cudaMemsetAsync(outputTensor, 0, out_bytes, stream);


    checkCUDNN(cudnnConvolutionForward(cudnn,
                                   &alpha,
                                   input_descriptor,
                                   inputTensor,
                                   kernel_descriptor,
                                   kernelTensor,
                                   convolution_descriptor,
                                   convolution_algorithm,
                                   d_workspace,
                                   workspace_bytes,
                                   &beta,
                                   output_descriptor,
                                   outputTensor));

}

ConvLayers convlayer1;
ConvLayers convlayer2;
ConvLayers convlayer3;
ConvLayers convlayer4;
ConvLayers convlayer5;


float img[BATCH_SIZE][224][224][3];
float output1[BATCH_SIZE][224][224][10];
float output2[BATCH_SIZE][224][224][30];
float output3[BATCH_SIZE][224][224][30];
float output4[BATCH_SIZE][224][224][10];
float output5[BATCH_SIZE][224][224][3];

class Kernel{

  public :
    cudaStream_t stream;
    cublasHandle_t cublas;
    cudnnHandle_t cudnn;
    int id;
    int layer;
    double tbRatio;
    // float ** twoDInput;
    // float ** twoDOutput;
    double execTime;
    double compTime;
    

    Kernel(){}
    
    Kernel(cudaStream_t stream,
        cublasHandle_t cublas,
        cudnnHandle_t cudnn,
        int layerIndex,
        double startTime,
        double deadTime,
        double tbratio,
        int id){
            this->stream = stream;
            this->cublas = cublas;
            this->cudnn = cudnn;
            this->layer = layerIndex;
            this->execTime = deadTime - startTime;
            this->compTime = LAYERS*COMP_TIME;
            this->tbRatio = tbratio;
            this->id = id;
        }
    
    Kernel(cudaStream_t stream,
        cublasHandle_t cublas,
        cudnnHandle_t cudnn,
        int layerIndex,
        float **twoDInput,
        double startTime,
        double deadTime,
        double tbratio,
        int id){
            this->stream = stream;
            this->cublas = cublas;
            this->cudnn = cudnn;
            this->layer = layerIndex;
            // this->twoDInput = twoDInput;
            this->execTime = deadTime - startTime;
            this->compTime = LAYERS*COMP_TIME;
            this->tbRatio = tbratio;
            this->id =id;
        }
    
    void update()
    {
        layer++;
        compTime -= COMP_TIME;
        // TODO update tbratio properly
        if(layer >= 7){
            tbRatio = 0.2;
            // twoDInput = twoDOutput;
        }
        // else
        // {
        //     fourDInput = fourDOutput;
        // }
    }


};

void execute(vector<Kernel> C);


typedef pair<int, Kernel> Node;

struct cmp
{
  bool operator()(const Node &A,const Node &B) const
  {
    if(A.first != B.first)
            return A.first < B.first;
        else
            return A.second.execTime - A.second.compTime < B.second.execTime - B.second.compTime;
  }
};

typedef set<Node, cmp> SET;

bool compareKernel(Kernel k1, Kernel k2){
    return k1.compTime < k2.compTime;
}

int getSlackTime(int compTime, int execTime){
    return execTime - compTime;
}



vector<Kernel> G;

SET enqueue(SET Queue, vector<Kernel> k){
    SET newQueue;
    for(SET::iterator it=Queue.begin();it!=Queue.end();it++){
        Kernel temp_kernel = it->second;
        temp_kernel.execTime -= EXEC_TIME;
        newQueue.insert({getSlackTime(temp_kernel.compTime,temp_kernel.execTime),temp_kernel});
    }
    for(int i=0;i<k.size();i++){
        if(k[i].layer == LAYERS+1)
            continue;
        k[i].execTime -= EXEC_TIME;
        cout<<"Enqueue ID: "<<k[i].id<<" "<<k[i].layer<<endl;
        newQueue.insert({getSlackTime(k[i].compTime, k[i].execTime), k[i]});
    }
    return newQueue;
}


SET dequeue(SET& Queue){
    SET newQueue;
    
    cout<<"Printing Queue"<<endl;
    cout<<Queue.size()<<endl;
    for(SET:: iterator it=Queue.begin();it!=Queue.end();it++){
        Kernel temp_kernel = it->second;
        cout<<"Dequeu ID: "<<temp_kernel.id<<" "<<temp_kernel.layer<<endl;
        temp_kernel.execTime -= COMP_TIME;
        newQueue.insert({getSlackTime(temp_kernel.compTime,temp_kernel.execTime),temp_kernel});
    }
    Queue.clear();
    vector<Kernel> C;
    if(G.size()>0){
        execute(G);
        cout<<"Execution size "<<G.size()<<endl;
    }
        
    for(int i=0;i<G.size();i++){
        cout<<"EXEC Layer: "<<G[i].layer<<endl;
        cout<<"Priority:"<<G[i].execTime - G[i].compTime<<endl;
        if(G[i].layer != LAYERS){
            G[i].update();
        }
    }
    newQueue = enqueue(newQueue, G);
    G.clear();
    Kernel temp = (*(newQueue.begin())).second;
    newQueue.erase(newQueue.begin());
    G.push_back(temp);
    double total = 0;
    total += temp.tbRatio;
    if(total < 1){
        for(SET::iterator it = newQueue.begin();it!=newQueue.end();){
            if((*it).second.tbRatio<1){
                Kernel temp2 = (*it).second;
                total += temp2.tbRatio;
                it = newQueue.erase(it);
                G.push_back(temp2);
            }
            else
            {
                it++;
            }
            cout<<"Total TBratio "<<total<<endl;
            if(total > 1)
                break;
        }
    }
    else
    {
        cout<<"Single kernel execution"<<endl;
        execute(G);
        for(int i=0;i<G.size();i++){
            cout<<"Priority:"<<G[i].execTime - G[i].compTime<<endl;
            if(G[i].layer != LAYERS){
                G[i].update();
            }
            else
                G[i].layer++;
        }
        newQueue = enqueue(newQueue, G);
        G.clear();

        return newQueue;
    }

    execute(G);
    cout<<"Execution size: "<<G.size()<<endl;
    for(int i=0;i<G.size();i++){
        cout<<"Priority:"<<G[i].execTime - G[i].compTime<<endl;
        if(G[i].layer != LAYERS){
            G[i].update();
        }
        else
        {
            G[i].layer++;
        }
    }
    newQueue = enqueue(newQueue, G);
    G.clear();
    return newQueue;   
}

void call_conv1(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float input[][224][224][3], float output[BATCH_SIZE][224][224][10])
{
    float *inputTensor;
	float *outputTensor;
	int inp_size = convlayer1.inDims.Height * convlayer1.inDims.Width * convlayer1.inDims.Channels * convlayer1.inDims.Batch * sizeof(float);
    cudaMalloc(&inputTensor, inp_size);
    cudaMemcpyAsync(inputTensor, input, inp_size, cudaMemcpyHostToDevice, stream);

    void *d_workspace;
    convlayer1.fwdProp(stream, CUDNN, inputTensor, outputTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch*sizeof(float);
    
    cudaMemcpyAsync(output, outputTensor, out_size, cudaMemcpyDeviceToHost, stream);

    cout << "CONV1 DONE" << endl;
}

void call_conv2(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float input[][224][224][10], float output[BATCH_SIZE][224][224][30])
{
    float *inputTensor;
	float *outputTensor;
	int inp_size = convlayer1.inDims.Height * convlayer1.inDims.Width * convlayer1.inDims.Channels * convlayer1.inDims.Batch * sizeof(float);
    cudaMalloc(&inputTensor, inp_size);
    cudaMemcpyAsync(inputTensor, input, inp_size, cudaMemcpyHostToDevice, stream);

    void *d_workspace;
    convlayer1.fwdProp(stream, CUDNN, inputTensor, outputTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch*sizeof(float);
    
    cudaMemcpyAsync(output, outputTensor, out_size, cudaMemcpyDeviceToHost, stream);

    cout << "CONV2 DONE" << endl;
}


void call_conv3(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float input[][224][224][30], float output[BATCH_SIZE][224][224][30])
{
    float *inputTensor;
	float *outputTensor;
	int inp_size = convlayer1.inDims.Height * convlayer1.inDims.Width * convlayer1.inDims.Channels * convlayer1.inDims.Batch * sizeof(float);
    cudaMalloc(&inputTensor, inp_size);
    cudaMemcpyAsync(inputTensor, input, inp_size, cudaMemcpyHostToDevice, stream);

    void *d_workspace;
    convlayer1.fwdProp(stream, CUDNN, inputTensor, outputTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch*sizeof(float);
    
    cudaMemcpyAsync(output, outputTensor, out_size, cudaMemcpyDeviceToHost, stream);

    cout << "CONV3 DONE" << endl;
}

void call_conv4(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float input[][224][224][30], float output[BATCH_SIZE][224][224][10])
{
    float *inputTensor;
	float *outputTensor;
	int inp_size = convlayer1.inDims.Height * convlayer1.inDims.Width * convlayer1.inDims.Channels * convlayer1.inDims.Batch * sizeof(float);
    cudaMalloc(&inputTensor, inp_size);
    cudaMemcpyAsync(inputTensor, input, inp_size, cudaMemcpyHostToDevice, stream);

    void *d_workspace;
    convlayer1.fwdProp(stream, CUDNN, inputTensor, outputTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch*sizeof(float);
    
    cudaMemcpyAsync(output, outputTensor, out_size, cudaMemcpyDeviceToHost, stream);

    cout << "CONV4 DONE" << endl;
}

void call_conv5(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float input[][224][224][10], float output[BATCH_SIZE][224][224][3])
{
    float *inputTensor;
	float *outputTensor;
	int inp_size = convlayer1.inDims.Height * convlayer1.inDims.Width * convlayer1.inDims.Channels * convlayer1.inDims.Batch * sizeof(float);
    cudaMalloc(&inputTensor, inp_size);
    cudaMemcpyAsync(inputTensor, input, inp_size, cudaMemcpyHostToDevice, stream);

    void *d_workspace;
    convlayer1.fwdProp(stream, CUDNN, inputTensor, outputTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch*sizeof(float);
    
    cudaMemcpyAsync(output, outputTensor, out_size, cudaMemcpyDeviceToHost, stream);

    cout << "CONV5 DONE" << endl;
}




void execute(vector<Kernel> C)
{
    
    for(int i=0; i<C.size();i++){
        Kernel K = C[i];
        if(K.layer == 1)
            call_conv1(convlayer1, K.stream, K.cudnn, img, output1);
        else if(K.layer == 2)
            call_conv2(convlayer2, K.stream, K.cudnn, output1, output2);
        else if(K.layer == 3)
            call_conv3(convlayer3, K.stream, K.cudnn, output2, output3);
        else if(K.layer == 4)
            call_conv4(convlayer4, K.stream, K.cudnn, output3, output4);
        else
            call_conv5(convlayer5, K.stream, K.cudnn, output4, output5);
    }
	
}

//channel, height, width
int main(){
  for(int i=0;i<BATCH_SIZE;++i){
    for(int j=0; j<224;++j){
      for(int k=0;k<224;++k){
        for(int l=0;l<3;++l){
          img[i][j][k][l]=0.001;
        }
      }
    }
  }

	convDim_t InputDims1 = setConvSpecs(224, 224, 3, BATCH_SIZE);
	kernelDim_t layerKernel1 = setKernelSpecs(0,3,3,1,1,1,1,1,1);
	convDim_t OutputDims1 = setConvSpecs(224, 224, 10, BATCH_SIZE);
	convlayer1.set(1, InputDims1, layerKernel1, OutputDims1);
	convlayer1.buildConvLayer();
	
	convDim_t InputDims2 = setConvSpecs(224, 224, 10, BATCH_SIZE);
	kernelDim_t layerKernel2 = setKernelSpecs(0,3,3,1,1,1,1,1,1);
	convDim_t OutputDims2 = setConvSpecs(224, 224, 30, BATCH_SIZE);
	convlayer2.set(2, InputDims2, layerKernel2, OutputDims2);
	convlayer2.buildConvLayer();

	convDim_t InputDims3 = setConvSpecs(224, 224, 30, BATCH_SIZE);
	kernelDim_t layerKernel3 = setKernelSpecs(0,3,3,1,1,1,1,1,1);
	convDim_t OutputDims3 = setConvSpecs(224, 224, 30, BATCH_SIZE);
	convlayer3.set(3, InputDims3, layerKernel3, OutputDims3);
	convlayer3.buildConvLayer();

	convDim_t InputDims4 = setConvSpecs(224, 224, 30, BATCH_SIZE);
	kernelDim_t layerKernel4 = setKernelSpecs(0,3,3,1,1,1,1,1,1);
	convDim_t OutputDims4 = setConvSpecs(224, 224, 10, BATCH_SIZE);
	convlayer4.set(4, InputDims4, layerKernel4, OutputDims4);
	convlayer4.buildConvLayer();

	convDim_t InputDims5 = setConvSpecs(224, 224, 10, BATCH_SIZE);
	kernelDim_t layerKernel5 = setKernelSpecs(0,3,3,1,1,1,1,1,1);
	convDim_t OutputDims5 = setConvSpecs(224, 224, 3, BATCH_SIZE);
	convlayer5.set(5, InputDims5, layerKernel5, OutputDims5);
	convlayer5.buildConvLayer();

	int nstreams = 4;
	cudaStream_t stream[nstreams];
	cudnnHandle_t cudnn[nstreams];
    cublasHandle_t cublas[nstreams];

  

	for(int i=0;i<nstreams;++i){
		cudaStreamCreate(&stream[i]);
		checkCUDNN(cudnnCreate(&cudnn[i]));
		cudnnSetStream(cudnn[i], stream[i]);
        cublasSetStream(cublas[i], stream[i]);
	}

for(int j=1;j<=nstreams;j++){
    vector<Kernel> inp;
    SET pq;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    
    for(int i=0;i<NUM_BATCHES;i++)
    {
          Kernel k(stream[i%j], cublas[i%j], cudnn[i%j], 1, 0, 30+i, 1.2, i);   
          inp.push_back(k);
    }
      pq = enqueue(pq, inp);
      while(!pq.empty()){
          pq = dequeue(pq);
      }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float elapsedtime;
    cudaEventElapsedTime(&elapsedtime, start, stop);
    cout<<"Num Streams"<<j << "ELAPSED TIME: " << elapsedtime << endl;
    cudaDeviceSynchronize();

}
    

    cout<<output1[0][0][0][0]<<endl;
    cout<<output2[0][0][0][0]<<endl;
    cout<<output3[0][0][0][0]<<endl;
    cout<<output4[0][0][0][0]<<endl;
    cout<<output5[0][0][0][0]<<endl;
	return 0;
}



'File written in /content/src/alex.cu'

In [ ]:
!nvcc /content/src/alex.cu `pkg-config --cflags --libs opencv` -lcudnn -lcublas -lopencv_imgcodecs -lopencv_imgproc -lopencv_core -pg -std=c++11 -o /content/alex

In [ ]:
!/content/alex 

Enqueue ID: 0 1
Enqueue ID: 1 1
Printing Queue
2
Dequeu ID: 0 1
Dequeu ID: 1 1
Single kernel execution
CONV1 DONE
Priority:10
Enqueue ID: 0 2
Printing Queue
2
Dequeu ID: 1 1
Dequeu ID: 0 2
Single kernel execution
CONV1 DONE
Priority:6
Enqueue ID: 1 2
Printing Queue
2
Dequeu ID: 0 2
Dequeu ID: 1 2
Single kernel execution
CONV2 DONE
Priority:3
Enqueue ID: 0 3
Printing Queue
2
Dequeu ID: 1 2
Dequeu ID: 0 3
Single kernel execution
CONV2 DONE
Priority:-1
Enqueue ID: 1 3
Printing Queue
2
Dequeu ID: 0 3
Dequeu ID: 1 3
Single kernel execution
CONV3 DONE
Priority:-4
Enqueue ID: 0 4
Printing Queue
2
Dequeu ID: 1 3
Dequeu ID: 0 4
Single kernel execution
CONV3 DONE
Priority:-8
Enqueue ID: 1 4
Printing Queue
2
Dequeu ID: 0 4
Dequeu ID: 1 4
Single kernel execution
CONV4 DONE
Priority:-11
Enqueue ID: 0 5
Printing Queue
2
Dequeu ID: 1 4
Dequeu ID: 0 5
Single kernel execution
CONV4 DONE
Priority:-15
Enqueue ID: 1 5
Printing Queue
2
Dequeu ID: 0 5
Dequeu ID: 1 5
Single kernel execution
CONV5 DONE
Priori

!